# AdaBoost 

In [105]:
import numpy as np
import pandas as pd

samples = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
labels = np.array([1, 1, 1, -1, -1, -1, 1, 1, 1, -1])

initial_threshold = 2.5
iterations = 6

weights = np.ones(len(samples)) / len(samples)

weights_per_iteration = [weights.copy()]  
alpha_per_iteration = [0]  
thresholds_per_iteration = [initial_threshold]  
def weak_classifier(x, theta):
    return np.where(x < theta, 1, -1)

def find_optimal_threshold(samples, labels, weights,iter):
    min_sample, max_sample = np.min(samples), np.max(samples)
    best_threshold = min_sample
    min_error = float('inf')
    
   
    threshold = min_sample
    while threshold <= max_sample:
        predictions = weak_classifier(samples, threshold)
        misclassified = (predictions != labels)
        error = np.sum(weights * misclassified)
        print(iter,threshold,error,misclassified,)
        
        if error < min_error:
            min_error = error
            best_threshold = threshold
        
        
        threshold += 0.5
    
    print(best_threshold,min_error)
            
    return best_threshold, min_error


e = []
pred = []
for i in range(1, iterations + 1):
    threshold, error = find_optimal_threshold(samples, labels, weights,i)
    thresholds_per_iteration.append(threshold)

    e.append(error)
    
    alpha = 0.5 * np.log((1 - error) / error) 
    alpha_per_iteration.append(alpha)
    
    predictions = weak_classifier(samples, threshold)
    pred.append([i,predictions]) 
    misclassified = (predictions != labels)
    
    for j in range(len(weights)):
        if misclassified[j]:
            weights[j] *= np.exp(alpha)  
        else:
            weights[j] *= np.exp(-alpha)  
    
    weights /= np.sum(weights)
    
    
    weights_per_iteration.append(weights.copy())



weights_df = pd.DataFrame(weights_per_iteration, columns=[f"x{i}" for i in range(len(samples))], index=[f"Iteration {i}" for i in range(iterations + 1)])
alpha_df = pd.DataFrame(alpha_per_iteration, columns=["Alpha"], index=[f"Iteration {i}" for i in range(iterations + 1)])
thresholds_df = pd.DataFrame(thresholds_per_iteration, columns=["Threshold"], index=[f"Iteration {i}" for i in range(iterations + 1)])



1 0 0.6 [ True  True  True False False False  True  True  True False]
1 0.5 0.5 [False  True  True False False False  True  True  True False]
1 1.0 0.5 [False  True  True False False False  True  True  True False]
1 1.5 0.4 [False False  True False False False  True  True  True False]
1 2.0 0.4 [False False  True False False False  True  True  True False]
1 2.5 0.30000000000000004 [False False False False False False  True  True  True False]
1 3.0 0.30000000000000004 [False False False False False False  True  True  True False]
1 3.5 0.4 [False False False  True False False  True  True  True False]
1 4.0 0.4 [False False False  True False False  True  True  True False]
1 4.5 0.5 [False False False  True  True False  True  True  True False]
1 5.0 0.5 [False False False  True  True False  True  True  True False]
1 5.5 0.6 [False False False  True  True  True  True  True  True False]
1 6.0 0.6 [False False False  True  True  True  True  True  True False]
1 6.5 0.5 [False False False  True

In [114]:
weights_df

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9
Iteration 0,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
Iteration 1,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.166667,0.166667,0.166667,0.071429
Iteration 2,0.045455,0.045455,0.045455,0.166667,0.166667,0.166667,0.106061,0.106061,0.106061,0.045455
Iteration 3,0.033333,0.033333,0.033333,0.122222,0.122222,0.122222,0.166667,0.166667,0.166667,0.033333
Iteration 4,0.026316,0.026316,0.026316,0.166667,0.166667,0.166667,0.131579,0.131579,0.131579,0.026316
Iteration 5,0.021739,0.021739,0.021739,0.137681,0.137681,0.137681,0.166667,0.166667,0.166667,0.021739
Iteration 6,0.018519,0.018519,0.018519,0.166667,0.166667,0.166667,0.141975,0.141975,0.141975,0.018519


# KNN

In [125]:
import scipy.io
import pandas as pd

mat_file_path = 'test_data.mat'
mat_data = scipy.io.loadmat(mat_file_path)

data_X = mat_data['X']
data_Y = mat_data['Y']

df_X = pd.DataFrame(data_X)
df_Y = pd.DataFrame(data_Y)

df_combined = pd.concat([df_X, df_Y], axis=1)


df_combined.columns = [f"X{i+1}" for i in range(df_X.shape[1])] + [f"Y{i+1}" for i in range(df_Y.shape[1])]


output_csv_path = 'test.csv'
df_combined.to_csv(output_csv_path, index=False)


In [205]:
import pandas as pd
import numpy as np
import math
from collections import Counter
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt



train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

y_test = test_data['Y'].iloc[:1000].tolist()

train = train_data.iloc[:6000].to_numpy()
test = test_data.iloc[:1000].to_numpy()

y_test1 = train_data['Y'].iloc[:1000].tolist()

test1 = train_data.iloc[:1000].to_numpy()



def euclidean_distance(sample1, sample2):
    return np.linalg.norm(sample1[:-1] - sample2[:-1])



def voting(array):
    second_elements = [row[1] for row in array]
    vote_counts = Counter(second_elements)
    vote = vote_counts.most_common(1)[0][0]
    return vote

def KNN_kernel(train_d,test_d,K):
	dist = []
	dist = [[euclidean_distance(train_d[i], test_d), train_d[i][-1]] for i in range(len(train_d))]
	soretd_dist = sorted(dist, key=lambda x: x[0])
	return voting(soretd_dist[:K])


K = [1, 9, 19, 29, 39, 49, 59, 69, 79, 89, 99]
error_list = []

for k in K:
	y_pred = []
	for t in test:
		y_pred.append(KNN_kernel(train,t,k))

	accuracy = accuracy_score(y_test, y_pred)
	error = 1 - accuracy

	print(error)
	
	error_list.append([k,error])
error_list1 = []
for k in K:
	y_pred = []
	for t in test1:
		y_pred.append(KNN_kernel(train,t,k))

	accuracy = accuracy_score(y_test1, y_pred)
	error = 1 - accuracy

	print(error)
	
	error_list1.append([k,error])


error_list = [[1, 0.014000000000000012], [9, 0.015000000000000013], [19, 0.028000000000000025], [29, 0.03500000000000003], [39, 0.040000000000000036], [49, 0.040000000000000036], [59, 0.04600000000000004], [69, 0.049000000000000044], [79, 0.051000000000000045], [89, 0.05400000000000005], [99, 0.05300000000000005]]
x_values = [point[0] for point in error_list]
y_values = [point[1] for point in error_list]

x_values1 = [point[0] for point in error_list1]
y_values1 = [point[1] for point in error_list1]


plt.plot(x_values, y_values, marker='o', linestyle='-', color='b')
plt.plot(x_values1, y_values1, marker='o', linestyle='-', color='r')
plt.xlabel("K value")
plt.ylabel("Error Rate")

plt.grid()


plt.savefig("plot.png", dpi=300)  
plt.show()

[1]